In [488]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.utils import shuffle
from scipy.sparse import coo_matrix
from sklearn.feature_selection import SelectKBest,chi2

In [764]:
# Construct training-set
co = pd.read_csv('../outputs/log-counts.tsv', sep='\t', encoding = "ISO-8859-1")
co = co.set_index('ensembl_Id')
tp = pd.read_csv('../outputs/dge/top_up.csv', sep=',', encoding = "ISO-8859-1")
dw = pd.read_csv('../outputs/dge/top_down.csv', sep=',', encoding = "ISO-8859-1")
ts = pd.read_csv("../outputs/camda-test-set-82.csv")
ts = ts.set_index('sample_id')

In [765]:
df = pd.DataFrame()

for i, b in enumerate(tp['symbol'].notnull()):
    if b:
        transcript = tp['transcriptId'].iloc[i]
        gene = tp['symbol'].iloc[i]
        if gene in list(ts.columns):
            df[gene] = co.loc[transcript]
        
for i, b in enumerate(dw['symbol'].notnull()):
    if b:
        transcript = dw['transcriptId'].iloc[i]
        gene = dw['symbol'].iloc[i]
        if gene in list(ts.columns):
            df[gene] = co.loc[transcript]

In [766]:
target = []
for i, _ in df.iterrows():
    if i.startswith('ss'):
        target.append(0)
    else:
        target.append(1)
        
df['target'] = target

In [767]:
df.shape

(32, 17)

In [768]:
## ANN ML ##
X = df.iloc[:, :16]
y = df.iloc[:, 16]

In [676]:
# Feature selection - ***SKIP***
X = SelectKBest(chi2, k=15).fit(X, y)
mask = X.get_support()
new_feat = []
el_feat = []
for bool, feature in zip(mask, df.columns):
    if bool:
        new_feat.append(feature)
    else:
        el_feat.append(feature)
print('The best features are:{}'.format(new_feat))
print('The eliminated features are:{}'.format(el_feat))

The best features are:['EVX2', 'NHLH2', 'PRSS12', 'POU6F2', 'MAPK15', 'RTL1', 'LGR5', 'DPY19L2P4', 'STRA6', 'CYP17A1', 'OR10AB1P', 'LRRTM3', 'HIST1H1E', 'NXPH3', 'MYL3']
The eliminated features are:['HOXD10', 'MYH14', 'GRIN3A', 'HS3ST5', 'NBAS', 'CRYAB', 'CMYA5', 'AMIGO2', 'EDIL3', 'UBC']


In [769]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [770]:
X_sparse = coo_matrix(X)
X, X_sparse, y = shuffle(X, X_sparse, y, random_state=0)

In [703]:
# Tuning Hyperparameters - SKIP
parameters = {'solver': ['adam','lbfgs','sgd'],
              'activation': ['identity', 'logistic', 'tanh', 'relu'],
              'max_iter': [10000], 
              'alpha': [0.01,0.1],
              'hidden_layer_sizes':[6,12,24,48,100],
              'random_state':[0,1,2,3,4]}

mlp = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1, cv=5,iid=False)
mlp = mlp.fit(X, y)
print(mlp.score(X, y))
print(mlp.best_params_)

0.875
{'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': 6, 'max_iter': 10000, 'random_state': 0, 'solver': 'sgd'}


In [771]:
# Fit ML
#mlp = MLPClassifier(hidden_layer_sizes=(10,),activation='relu',solver='sgd',max_iter=7000, random_state=2,alpha=0.1)
#mlp = MLPClassifier(hidden_layer_sizes=(24,24,24),activation='relu',solver='sgd',max_iter=10000,random_state=3,alpha=0.01)
#mlp = MLPClassifier(hidden_layer_sizes=(6,),activation='identity',solver='adam',max_iter=10000,random_state=2,alpha=0.01)
mlp = MLPClassifier(hidden_layer_sizes=(6,),activation='relu',solver='sgd',max_iter=10000,random_state=0,alpha=0.1)# test-set-1 82%
#mlp = MLPClassifier(hidden_layer_sizes=(24,),activation='relu',solver='sgd',max_iter=10000,random_state=2,alpha=0.01)# test-set-1 82%

In [772]:
mlp = mlp.fit(X, y)

In [773]:
# Test ML
X_test = ts.iloc[:, :16]
y_test = ts.iloc[:, 16]

In [774]:
X_test = scaler.transform(X_test)

In [775]:
predict_test = mlp.predict(X_test)
print(confusion_matrix(y_test, predict_test))
print(accuracy_score(y_test, predict_test))
print(classification_report(y_test, predict_test))

[[37  5]
 [ 6 13]]
0.819672131147541
              precision    recall  f1-score   support

           0       0.86      0.88      0.87        42
           1       0.72      0.68      0.70        19

   micro avg       0.82      0.82      0.82        61
   macro avg       0.79      0.78      0.79        61
weighted avg       0.82      0.82      0.82        61



# Another

In [776]:
## SVM ML ##
from libsvm.svmutil import *
from sklearn.datasets import dump_svmlight_file

In [779]:
dump_svmlight_file(X,y,'../outputs/svm/camda.svm', zero_based=False)

In [780]:
dump_svmlight_file(X_test,y_test,'../outputs/svm/camda-test.svm', zero_based=False)

In [781]:
# Use these command lines to scale and tune hyperparameters
# ./libsvm-3.23/svm-scale -l 0 -u 1 -s t.scale camda.svm > camda.scale
# ./libsvm-3.23/svm-scale -r t.scale camda-test.svm > camda-test.scale
# ./libsvm-3.23/tools/grid.py camda.scale

In [834]:
y, X = svm_read_problem("../outputs/svm/camda.scale")

In [836]:
cross_val_accuracy = svm_train(y, X, '-c 32.0 -g 0.0078125 -t 2 -v 5')

Cross Validation Accuracy = 78.125%


In [820]:
y_test, X_test = svm_read_problem("../outputs/svm/camda-test.scale")

In [827]:
model = svm_train(y, X, '-c 32.0 -t 2 -g 0.0078125')

In [828]:
p_label, p_acc, p_val = svm_predict(y_test, X_test, model)

Accuracy = 78.6885% (48/61) (classification)


In [805]:
print(classification_report(y_test, p_label))

              precision    recall  f1-score   support

         0.0       0.87      0.81      0.84        42
         1.0       0.64      0.74      0.68        19

   micro avg       0.79      0.79      0.79        61
   macro avg       0.75      0.77      0.76        61
weighted avg       0.80      0.79      0.79        61



# Another

In [108]:
## RandomForest - Feature selection ##
from sklearn.ensemble import RandomForestClassifier

In [157]:
# Hyperparameter tuning
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 40)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(1, 20, num=20)]
max_depth.append(None)
min_samples_split = [2,3,4,5]
min_samples_leaf = [1,2,3]
bootstrap = [True, False]
random_state = [int(x) for x in np.linspace(0,50, num=50)]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'random_state': random_state}

In [158]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 5, verbose=2, n_jobs = -1, iid=False)
mlp = rf_random.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    4.0s finished


In [159]:
rf_random.best_params_

{'random_state': 20,
 'n_estimators': 39,
 'min_samples_split': 3,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 4,
 'bootstrap': True}

In [167]:
#mlp = rf_random.best_estimator_ 
#mlp = RandomForestClassifier(n_jobs=-1,random_state=24,n_estimators=36,
#                            min_samples_split=3,min_samples_leaf=3,
#                            max_features='auto',max_depth=9,bootstrap=True)

mlp = RandomForestClassifier(n_jobs=-1,random_state=0,n_estimators=100,
                            min_samples_split=4,min_samples_leaf=4,
                            max_features='sqrt',max_depth=7,bootstrap=True)

# X and y are from ANN (Not SVM)
cross_val_score(mlp, X, y, cv=5)

array([0.71428571, 0.71428571, 0.83333333, 0.83333333, 1.        ])

In [168]:
mlp = mlp.fit(X, y)
preds = mlp.predict(X_test)
mlp.score(X_test, y_test)

0.7540983606557377

In [169]:
importances = list(mlp.feature_importances_)
feature_list = list(df.columns)
feature_importances = [(f, round(i, 2)) for f, i in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:10} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: DPY19L2P4  Importance: 0.14
Variable: EDIL3      Importance: 0.1
Variable: STRA6      Importance: 0.08
Variable: LGR5       Importance: 0.07
Variable: LRRTM3     Importance: 0.07
Variable: NXPH3      Importance: 0.07
Variable: MYL3       Importance: 0.07
Variable: UBC        Importance: 0.07
Variable: RTL1       Importance: 0.06
Variable: NHLH2      Importance: 0.04
Variable: HS3ST5     Importance: 0.04
Variable: HIST1H1E   Importance: 0.03
Variable: EVX2       Importance: 0.02
Variable: PRSS12     Importance: 0.02
Variable: MAPK15     Importance: 0.02
Variable: CYP17A1    Importance: 0.02
Variable: CRYAB      Importance: 0.02
Variable: AMIGO2     Importance: 0.02
Variable: GRIN3A     Importance: 0.01
Variable: NBAS       Importance: 0.01
Variable: POU6F2     Importance: 0.0
Variable: HOXD10     Importance: 0.0
Variable: OR10AB1P   Importance: 0.0
Variable: MYH14      Importance: 0.0
Variable: CMYA5      Importance: 0.0


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]